In [1]:
%%capture
!pip install ultralytics==8.1.33

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import re
import matplotlib.patches as patches
from ultralytics import YOLO
import os
import yaml
from tqdm import tqdm
import shutil
import wandb
import glob
import json
from sklearn.metrics import mean_absolute_error
import matplotlib.image as mpimg
import glob
from sklearn.model_selection import StratifiedKFold
exist=False
from sklearn.model_selection import GroupKFold
import torch
import gc

In [3]:
import zipfile
local_zip = 'test.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('test')
zip_ref.close()

In [4]:
local_zip = 'train.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('images')
zip_ref.close()

In [5]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(42)

In [6]:
def get_unique_image_shapes_from_dataframe(CFG,df):
    unique_shapes = []
    for index, row in tqdm(df.iterrows()):
        image_path = CFG.images_path + row[CFG.image_id] + ".png"
        with Image.open(image_path) as img:
            shape = img.size[::-1]  
            if shape not in unique_shapes:
                unique_shapes.append(shape)
            df.iloc[index,7] = shape[0]
    return unique_shapes


def create_data_folder(destination_folder, df):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    unique_images = df[CFG.image_id].unique()
    for img in unique_images:
        img_path = os.path.join(CFG.images_path, img + '.png')
        shutil.copy(img_path, destination_folder)
        
        
def convert_bbox_to_string(row):
    x_max = row['Xmax']
    x_min = row['Xmin']
    y_min = row['Ymin'] 
    y_max = row['Ymax'] 
    xc = int(np.round((x_min + x_max) / 2))
    yc = int(np.round((y_min + y_max) / 2))
    w = x_max - x_min
    h = y_max - y_min
    box = [xc/row['shape'], yc/row['shape'], w/row['shape'], h/row['shape']]
    box = [f"{i:.4g}" for i in box]
    return " ".join(box)


def create_label_folder(label_dir, dataframe):
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
    unique_images = dataframe[CFG.image_id].unique()
    for i,img in zip(tqdm(range(len(unique_images))), unique_images):
        df = dataframe[dataframe[CFG.image_id] == img]
        output_lines = []
        for index, row in df.iterrows():
            category_id = int(row[CFG.category_id]) 
            category_id=reversed_labels[category_id]
            bbox_string = convert_bbox_to_string(row)
            output_lines.append(f'{category_id} {bbox_string}')
        with open(f"{label_dir}/{img}.txt", 'w') as f:
            for line in output_lines:
                f.write(line + '\n')
               
            
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as e:
            print("Error reading YAML:", e)
            return None

        
def print_yaml_data(data):
    formatted_yaml = yaml.dump(data, default_style=False)
    print(formatted_yaml)
    

def display_image(file_path, print_info=False, hide_axis=True):
    img = mpimg.imread(file_path)
    print(img.shape)
    plt.imshow(img)
    if print_info:
        print(f"File path: {file_path}")
        print(f"Image shape: {img.shape}")
    if hide_axis:
        plt.axis('off')
    plt.show()
    
    
def convert_id (id):
    return 'ID' + '_'+ ((6 - len(str(id)))*'0') + str(id)

In [7]:
labels={0:11,1:12,2:13,3:14,4:15,5:16,6:17,7:18,8:21,9:22,10:23,11:24,12:25,13:26,14:27,15:28,16:31,17:32,18:33,19:34,20:35,21:36,22:37,23:38,24:41,25:42,26:43,27:44,28:45,29:46,30:47,31:48}
reversed_labels = {value: key for key, value in labels.items()}

In [8]:
class CFG:
    train_path="Train.csv"
    test_path="Test.csv"
    train_dest_dir = "/notebooks/train"
    val_dest_dir = "/notebooks/val"
    train_destination_folder = "/notebooks/train/images"
    val_destination_folder = "/notebooks/val/images"
    train_destination_labels= "/notebooks/train/labels"
    val_destination_labels= "/notebooks/val/labels"
    images_path="images/train/"
    yaml="data.yaml"
    image_id = 'Image_ID' 
    category_id='ToothClass'
    num_classes = 32
    names=[i for i in range(32)]
    model='yolov8m.pt'
    task='detect'
    EPOCHS = 60
    BATCH_SIZE = 4
    OPTIMIZER = "auto"
    SEED = 42
    DEVICE = [0]
    VERBOSE = False
    RESUME = False
    PATIENCE = 5
    img_size=512
    lr0=0.018
    momentum=0.947
    weight_decay=0.0005
    close_mosaic=3

In [9]:
train = pd.read_csv(CFG.train_path)
test = pd.read_csv(CFG.test_path)
train.head()

,Image_ID,ToothClass,Xmin,Ymin,Xmax,Ymax
0,0,33,284,315,325,361
1,0,37,372,119,448,193
2,0,47,40,114,113,186
3,0,35,325,236,379,287
4,0,41,200,362,237,411


In [10]:
set_seed(42)
folds = GroupKFold(n_splits=5)
train['fold'] = -1
for i,(train_index, test_index) in enumerate(folds.split(train,train[CFG.category_id], groups=train[CFG.image_id])): 
    train.loc[test_index,'fold'] = i

In [11]:
train['Image_ID'] = [convert_id(id) for id in  list(train['Image_ID'])]

In [12]:
train['shape']=0
unique_shapes = get_unique_image_shapes_from_dataframe(CFG,train)

16013it [00:02, 6757.74it/s]


In [13]:
unique_images_test = test['Image_ID'].unique()
from datetime import datetime
timestamp = datetime.now().strftime('%d-%m-%Y-%H-%M-%S')
set_seed(42)
sub=pd.DataFrame(columns=["Image_ID"])
score=[]
subs = []
for fold in range(5) :
    sub_data = pd.DataFrame(columns=["Image_ID", "class",'confidence','ymin','xmin','ymax','xmax'])
    print(f'--------------------------------Training Fold {fold+1}/5---------------------------------')
    NAME = f"Arm_Yolo_{fold}"
    train_df = train[train.fold!=fold].reset_index()
    val_df = train[train.fold==fold].reset_index()
    if exist:
        shutil.rmtree(CFG.train_dest_dir) 
        shutil.rmtree(CFG.val_dest_dir) 
    exist=True
    create_data_folder(CFG.train_destination_folder, train_df)
    create_data_folder(CFG.val_destination_folder, val_df)
    create_label_folder(CFG.train_destination_labels, train_df)
    create_label_folder(CFG.val_destination_labels, val_df)
    dict_file = {
        'train': CFG.train_dest_dir,
        'val' : CFG.val_dest_dir,
        'nc': CFG.num_classes,
        'names': CFG.names
        }
    with open(CFG.yaml, 'w+') as file:
        yaml.dump(dict_file, file)
    yaml_data = read_yaml_file(CFG.yaml)
    if yaml_data:
        print_yaml_data(yaml_data)
    model = YOLO(CFG.model)
    model.train(
        data = CFG.yaml,
        project=timestamp,
        save=True,
        task = CFG.task,
        imgsz = CFG.img_size,
        epochs = CFG.EPOCHS,
        pretrained=True,
        batch = CFG.BATCH_SIZE,
        optimizer = CFG.OPTIMIZER,
        patience = CFG.PATIENCE,
        name =  f"Arm_Yolo_{fold}",
        seed = CFG.SEED,
        val = True,
        workers=8,
        resume = CFG.RESUME,
        device = CFG.DEVICE,
        verbose =  CFG.VERBOSE,
        lr0=CFG.lr0,
        momentum=CFG.momentum,
        weight_decay=CFG.weight_decay,
        close_mosaic=CFG.close_mosaic,
        hsv_h=0,
        hsv_s=0,
        hsv_v=0,
        translate=0,
        scale=0,
        shear=0,
        perspective=0,
        flipud=0,
        fliplr=0,
        bgr=0,
        mosaic=0,
        mixup=0,
        erasing=0,
        crop_fraction=0
    )
    model.export(
        format = 'onnx', 
        imgsz = CFG.img_size,
        half = False,
        int8 = False,
        simplify = False,
        nms = False,
    )
    for row in tqdm(unique_images_test, total=len(unique_images_test)):
        pred = model.predict(f"test/test/{row}.png", imgsz=CFG.img_size)
        d = {}
        d['Image_ID'] = row
        l= pred[0].boxes.cls.cpu().numpy().tolist()
        d['confidence'] = pred[0].boxes.conf.cpu().numpy().tolist()
        d['class']=[labels[i] for i in l]
        d['ymin'] = [l[1] for l in pred[0].boxes.xyxy.cpu().numpy().tolist()]
        d['xmin'] = [l[0] for l in pred[0].boxes.xyxy.cpu().numpy().tolist()]
        d['ymax'] = [l[3] for l in pred[0].boxes.xyxy.cpu().numpy().tolist()]
        d['xmax'] = [l[2] for l in pred[0].boxes.xyxy.cpu().numpy().tolist()]
        pred_df = pd.DataFrame(d)
        pred_df.reset_index(drop=True, inplace=True)  
        sub_data = pd.concat([sub_data, pred_df], axis=0, ignore_index=True)
    subs.append(sub_data)

--------------------------------Training Fold 1/5---------------------------------


100%|██████████| 240/240 [00:00<00:00, 853.70it/s]


names:
- 0
- 1
- 2
- 3
- 4
- 5
- 6
- 7
- 8
- 9
- 10
- 11
- 12
- 13
- 14
- 15
- 16
- 17
- 18
- 19
- 20
- 21
- 22
- 23
- 24
- 25
- 26
- 27
- 28
- 29
- 30
- 31
nc: 32
train: /notebooks/train
val: /notebooks/val

New https://pypi.org/project/ultralytics/8.2.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.33 🚀 Python-3.11.7 torch-2.1.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=60, time=None, patience=5, batch=4, imgsz=512, save=True, save_period=-1, cache=False, device=[0], workers=8, project=19-05-2024-11-01-57, name=Arm_Yolo_0, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=3, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.

2024-05-19 11:02:02.344895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 11:02:02.344955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 11:02:02.346204: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=32

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytic

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:04<00:00,  6.70it/s]


                   all        240       3202      0.954      0.913      0.963      0.914
Speed: 0.3ms preprocess, 14.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to 19-05-2024-11-01-57/Arm_Yolo_4


lr/pg0,▁▅████▇▇▇▇▇
lr/pg1,▁▅████▇▇▇▇▇
lr/pg2,▁▅████▇▇▇▇▇
metrics/mAP50(B),▁▆▆▇▇█▇██▇█
metrics/mAP50-95(B),▁▆▆▇▇█▇████
metrics/precision(B),▅▁▆█▄▇▆██▇▇
metrics/recall(B),▁▇▅▅█▇█▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▄▄▃▃▃▂▂▁▁


Ultralytics YOLOv8.1.33 🚀 Python-3.11.7 torch-2.1.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
Model summary (fused): 218 layers, 25858288 parameters, 0 gradients, 78.8 GFLOPs

PyTorch: starting from '19-05-2024-11-01-57/Arm_Yolo_4/weights/best.pt' with input shape (1, 3, 512, 512) BCHW and output shape(s) (1, 36, 5376) (49.6 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 1.4s, saved as '19-05-2024-11-01-57/Arm_Yolo_4/weights/best.onnx' (98.8 MB)

Export complete (1.9s)
Results saved to /notebooks/19-05-2024-11-01-57/Arm_Yolo_4/weights
Predict:         yolo predict task=detect model=19-05-2024-11-01-57/Arm_Yolo_4/weights/best.onnx imgsz=512  
Validate:        yolo val task=detect model=19-05-2024-11-01-57/Arm_Yolo_4/weights/best.onnx imgsz=512 data=data.yaml  
Visualize:       https://netron.app


100%|██████████| 600/600 [00:09<00:00, 65.47it/s]


In [93]:
sub_data = pd.DataFrame(columns=["Image_ID", "class",'confidence','ymin','xmin','ymax','xmax'])
sub0 = subs[0]
sub1 = subs[1]
sub2 = subs[2]
sub3 = subs[3]
sub4 = subs[4]

In [94]:
sub_data = pd.concat([sub_data, sub0])
sub_data = pd.concat([sub_data, sub1])
sub_data = pd.concat([sub_data, sub2])
sub_data = pd.concat([sub_data, sub3])
sub_data = pd.concat([sub_data, sub4])

In [95]:
sub_data['Numeric_ID'] = sub_data['Image_ID'].str.extract(r'(\d+)').astype(int)

In [96]:
sorted_df = sub_data.sort_values(by='Numeric_ID')

In [97]:
sorted_df = sorted_df.reset_index(drop=True)

In [98]:
grouped_df = sorted_df.groupby(['Image_ID', 'class']).agg({
    'confidence': 'median',
    'ymin': 'median',
    'xmin': 'median',
    'ymax': 'median',
    'xmax': 'median',
}).reset_index()

In [99]:
sub_data = grouped_df.copy()

In [100]:
sub_data['class']=sub_data['class'].astype(int)
sub_data['class']=sub_data['class'].astype(str)
sub_data['class']='class_'+sub_data['class']

In [101]:
sub_data['xmax']=sub_data['xmax'].astype(int)
sub_data['ymax']=sub_data['ymax'].astype(int)
sub_data['xmin']=sub_data['xmin'].astype(int)
sub_data['ymin']=sub_data['ymin'].astype(int)

In [102]:
sub_data

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_001200,class_31,0.928150,350,256,391,293
1,ID_001200,class_32,0.976435,334,282,378,321
2,ID_001200,class_33,0.976308,303,308,352,349
3,ID_001200,class_34,0.969269,272,334,319,378
4,ID_001200,class_35,0.975305,228,344,278,396
...,...,...,...,...,...,...,...
8132,ID_001799,class_23,0.994975,319,120,372,167
8133,ID_001799,class_24,0.961857,280,97,328,154
8134,ID_001799,class_25,0.950104,239,75,287,134
8135,ID_001799,class_26,0.980059,166,45,249,123


In [103]:
sub_data.to_csv('subfinale1.csv',index=False)